In [1]:
!pip install anywidget
!pip install -U ipykernel

In [1]:
import altair as alt 
alt.data_transformers.enable('json')
import pandas as pd

In [12]:
baby_names = pd.read_csv('dpt2020.csv', sep = ";")
baby_names['annais'] = pd.to_numeric(baby_names['annais'], errors = "coerce", downcast = "integer")
baby_names['preusuel'] = baby_names['preusuel'].str.casefold()
print(len(baby_names))
baby_names

3727553


,sexe,preusuel,annais,dpt,nombre
0,1,_prenoms_rares,1900.0,02,7
1,1,_prenoms_rares,1900.0,04,9
2,1,_prenoms_rares,1900.0,05,8
3,1,_prenoms_rares,1900.0,06,23
4,1,_prenoms_rares,1900.0,07,9
...,...,...,...,...,...
3727548,2,zya,2018.0,59,3
3727549,2,zya,NaN,XX,264
3727550,2,zyna,2013.0,93,3
3727551,2,zyna,NaN,XX,59


In [21]:
# Bin the years into 5-year intervals
baby_names['annais_binned'] = (baby_names['annais'] // 5) * 5

# Group by name and binned years, then aggregate
baby_names_agg = baby_names.groupby(['preusuel', 'annais_binned', 'sexe'])['nombre'].sum().unstack(fill_value=0).reset_index()

# Rename columns
baby_names_agg.columns = ['preusuel', 'annais', 'pop_m', 'pop_f']  # Assuming '1' is male and '2' is female
baby_names_formatted = baby_names_agg[['preusuel', 'annais', 'pop_m', 'pop_f']]

baby_names_formatted

,preusuel,annais,pop_m,pop_f
0,_prenoms_rares,1900.0,6552,8780
1,_prenoms_rares,1905.0,7398,9221
2,_prenoms_rares,1910.0,7961,9811
3,_prenoms_rares,1915.0,5391,6808
4,_prenoms_rares,1920.0,9056,11005
...,...,...,...,...
79530,ïssa,2015.0,3,0
79531,ömer,1995.0,3,0
79532,ömer,2010.0,38,0
79533,ömer,2015.0,126,0


In [24]:
search_input = alt.param(
    value='',
    bind=alt.binding(
        input='search',
        placeholder="Name",
        name='Name: ',
    )
)

base = alt.Chart(baby_names_formatted).transform_filter(
    alt.FieldEqualPredicate(field='preusuel', equal=search_input)
).add_params(
    search_input
).properties(height=400)

chart_m = base.mark_bar(color='#7CB518').encode(
    x=alt.X('pop_m:Q',
            title='Male',
            sort='descending',
            scale=alt.Scale()),
    y=alt.Y('annais:N', axis=None, sort='descending')
)

middle = base.encode(
    y=alt.Y('annais:O', axis=None, sort='descending'),
    text=alt.Text('annais:O'),
).mark_text().properties(width=20)

chart_f = base.mark_bar(color="#F3DE2C").encode(
    x=alt.X('pop_f:Q',
            title='Female',
            sort='ascending',
            scale=alt.Scale()),
    y=alt.Y('annais:N', axis=None, sort='descending')
)

chart_m | chart_f
chart = alt.concat(chart_m, middle, chart_f, spacing=5)
 
jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'concat': [{'mark': …